＊列によっては：文字型になっているから, それを数値で置き換える.

In [45]:
import os
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif

In [104]:
#データの読み込み.
currentDir = os.getcwd()
df = pd.read_csv(currentDir + "/DATA/train.csv")
df
#print(df.isnull().sum()) # Age:177, Cabin682, Embarked:2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [47]:
train = df.copy()
train.drop(["Name", "Ticket", "Cabin"], axis = 1, inplace = True)
train["Age"].fillna(train["Age"].mean(), inplace=True)
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True) #最頻値[0]必要なんでや.
train["Sex"].replace("male", 0, inplace=True)
train["Sex"].replace("female", 1, inplace = True)
train["Embarked"].replace("Q", 0, inplace=True) #なぜEmbarkedの整数型が少数なの？
train["Embarked"].replace("C", 1, inplace=True)
train["Embarked"].replace("S", 2, inplace=True)
train
#print(train.isnull().sum())

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.000000,1,0,7.2500,2
1,2,1,1,1,38.000000,1,0,71.2833,1
2,3,1,3,1,26.000000,0,0,7.9250,2
3,4,1,1,1,35.000000,1,0,53.1000,2
4,5,0,3,0,35.000000,0,0,8.0500,2
...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000,2
887,888,1,1,1,19.000000,0,0,30.0000,2
888,889,0,3,1,29.699118,1,2,23.4500,2
889,890,1,1,0,26.000000,0,0,30.0000,1


In [79]:
col_explain = ["PassengerId", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = train[col_explain]
y = train["Survived"]
model  = LogisticRegression(solver='lbfgs', max_iter = 500)
for k in range(len(col_explain)):
    selected_explanatory_var = []
    selector = SelectKBest(score_func=f_classif, k=k+1)
    selector.fit(X,y)
    mask = selector.get_support()
    a = np.where(mask==True)
    for i in range(len(a[0])):
        selected_explanatory_var.append(col_explain[a[0][i]])
    #ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
    # max_iter = 100 => max_iter = 500
    result = model.fit(train[selected_explanatory_var], y)
    print("k=", i+1, selected_explanatory_var)
    #print("係数",result.coef_, "切片",result.intercept_)
    print(model.score(train[selected_explanatory_var],y))

k= 1 ['Sex']
0.7867564534231201
k= 2 ['Pclass', 'Sex']
0.7867564534231201
k= 3 ['Pclass', 'Sex', 'Fare']
0.7867564534231201
k= 4 ['Pclass', 'Sex', 'Fare', 'Embarked']
0.7845117845117845
k= 5 ['Pclass', 'Sex', 'Parch', 'Fare', 'Embarked']
0.7878787878787878
k= 6 ['Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Embarked']
0.7968574635241302
k= 7 ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
0.8047138047138047
k= 8 ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
0.8013468013468014


In [96]:
#k=7が決定係数が一番良い. k= 7 ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
model.fit(train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']], y)
df_test = pd.read_csv(currentDir + "/DATA/test.csv")
valid = df_test.copy()
#df_test.isnull().sum() =>Age86, Fare1, Cabin327
valid.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)
valid["Age"].fillna(valid["Age"].mean(), inplace=True)
valid["Fare"].fillna(valid["Fare"].mean(), inplace=True)
valid["Sex"].replace("male",   0, inplace = True)
valid["Sex"].replace("female", 1, inplace = True)
valid["Embarked"].replace("Q", 0, inplace=True) #なぜEmbarkedの整数型が少数なの？
valid["Embarked"].replace("C", 1, inplace=True)
valid["Embarked"].replace("S", 2, inplace=True)
valid

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.50000,0,0,7.8292,0
1,3,1,47.00000,1,0,7.0000,2
2,2,0,62.00000,0,0,9.6875,0
3,3,0,27.00000,0,0,8.6625,2
4,3,1,22.00000,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,0,30.27259,0,0,8.0500,2
414,1,1,39.00000,0,0,108.9000,1
415,3,0,38.50000,0,0,7.2500,2
416,3,0,30.27259,0,0,8.0500,2


In [101]:
predict = model.predict(valid)
submit_csv = pd.concat([df_test['PassengerId'], pd.Series(predict)], axis=1)
submit_csv.columns = ['PassengerId', 'Survived']
submit_csv.to_csv('./submition.csv', index=False)

In [ ]:
#欠損値を持つ列名の取得.
rack_colmname = []
for i in range(len(df.columns)):
    colname = df.columns[i]
    if(df[colname].isnull().any()==True):
        rack_colmname.append(colname)
print(rack_colmname)

In [ ]:
#ランダムフォレストのモデル.
RandomForest = RandomForestClassifier(
    bootstrap=True,      #ブートストラップサンプルをするかどうか.
    criterion='mse',     #control how the decision tree algorithm splits nodes:
    max_depth=None,      #決定木の深さの最大値.
    max_features='auto', #defalut,最適な分割をするために考慮する特徴量の数
    max_leaf_nodes=None, #default,生成される木の最大の葉の数.
    min_impurity_split=1e-07, #not, default:0, 
    min_samples_leaf=1,       #
    min_samples_split=2,      #サイトによって違う？このパラメータ値以上のデータ数を持たない分岐先は,それ以上の条件分岐がされず,葉となる.
    min_weight_fraction_leaf=0.0, #default, 外れ値??
    n_estimators=10,   #not, default = 100
    n_jobs=1,          #defalut, 並列処理するジョブの数, 
    oob_score=True,    #not, defalut:false, 汎化性能を検証するためにout-of-bag samplesを使うかどうか.
    random_state=None, #
    verbose=0,         #defalut, fittingとpredicttinのときの冗長性.
    warm_start=False   #defalut, ??
)